In [2]:
import re
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, LSTM
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm, linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from sklearn.decomposition import NMF
from scipy.sparse import vstack
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
df = pd.read_csv("data.csv")

X = np.array(df["Görüş"].values.astype('U')) 
df["Durum"].replace({"Olumlu": 1, "Olumsuz": 0}, inplace=True)
y = np.array(df["Durum"].values)

In [6]:
count = CountVectorizer()
X = count.fit_transform(X)

print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [8]:
FullMatrix=vstack([X_train, X_test])

### NMF ile veri dönüşümü

In [9]:
model = NMF(n_components=200, init='random', random_state=0)
W = model.fit_transform(FullMatrix)
H = model.components_

print(W.shape)
print(H.shape)

(8348, 200)
(200, 27155)


In [19]:
TESTvector=W[0:2681,:]
TRAINvector=W[2681:7930,:]

TestOUTPUT = y_train[:2681]
TrainOUTPUT = y_train[2681:]

### SVM

In [11]:
SVM = svm.SVC()
SVM.fit(TRAINvector,TrainOUTPUT)

predictions_SVM = SVM.predict(TESTvector)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, TestOUTPUT)*100)

SVM Accuracy Score ->  81.79783662812383


### Yapay Sinir Ağı

In [47]:
ysa = Sequential()
ysa.add(Dense(72, input_shape = (TRAINvector.shape)))
ysa.add(Activation("relu"))
ysa.add(Dropout(0.2))
ysa.add(Dense(48))
ysa.add(Activation("relu"))
ysa.add(Dense(1))
ysa.add(Activation("sigmoid"))

ysa.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

ysa.summary()

ysa.fit(TRAINvector,TrainOUTPUT, epochs=10, batch_size=16, validation_data=(TESTvector, TestOUTPUT))

test_loss, test_acc = ysa.evaluate(TESTvector, TestOUTPUT)
print("Loss -> ", test_loss, "Accuracy -> ", test_acc)

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 5249, 1, 72)       14472     
_________________________________________________________________
activation_6 (Activation)    (None, 5249, 1, 72)       0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 5249, 1, 72)       0         
_________________________________________________________________
dense_67 (Dense)             (None, 5249, 1, 48)       3504      
_________________________________________________________________
activation_7 (Activation)    (None, 5249, 1, 48)       0         
_________________________________________________________________
dense_68 (Dense)             (None, 5249, 1, 1)        49        
_________________________________________________________________
activation_8 (Activation)    (None, 5249, 1, 1)      

### Verileri LSTM'ye Uygun Hale Getirme

In [20]:
TRAINvector = np.reshape(TRAINvector, (TRAINvector.shape[0], 1, TRAINvector.shape[1]))
TESTvector = np.reshape(TESTvector, (TESTvector.shape[0], 1, TESTvector.shape[1]))

### LSTM

In [52]:
model = Sequential()
model.add(LSTM(96, activation='relu', recurrent_dropout = 0.2, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(48, activation='relu', recurrent_dropout = 0.2))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(TRAINvector,TrainOUTPUT, epochs=15, batch_size=32, validation_split=0.1)

loss, acc = model.evaluate(TESTvector, TestOUTPUT)
print("Loss -> ", loss, "Accuracy -> ", acc)

Epoch 1/15
148/148 [==============================] - 1s 9ms/step - loss: 0.6888 - accuracy: 0.5699 - val_loss: 0.6781 - val_accuracy: 0.7162
Epoch 2/15
148/148 [==============================] - 1s 5ms/step - loss: 0.6343 - accuracy: 0.7697 - val_loss: 0.5774 - val_accuracy: 0.8133
Epoch 3/15
148/148 [==============================] - 1s 4ms/step - loss: 0.5029 - accuracy: 0.8135 - val_loss: 0.4173 - val_accuracy: 0.8590
Epoch 4/15
148/148 [==============================] - 1s 4ms/step - loss: 0.4110 - accuracy: 0.8398 - val_loss: 0.3615 - val_accuracy: 0.8514
Epoch 5/15
148/148 [==============================] - 1s 5ms/step - loss: 0.3895 - accuracy: 0.8448 - val_loss: 0.3374 - val_accuracy: 0.8629
Epoch 6/15
148/148 [==============================] - 1s 4ms/step - loss: 0.3712 - accuracy: 0.8486 - val_loss: 0.3372 - val_accuracy: 0.8495
Epoch 7/15
148/148 [==============================] - 1s 4ms/step - loss: 0.3579 - accuracy: 0.8527 - val_loss: 0.3223 - val_accuracy: 0.8590
Epoch 